In [49]:
import pandas as pd
import numpy as np

In [50]:
data = pd.read_csv('Data.csv')
data["Date"] = pd.to_datetime(data["Date"])

In [51]:
data.dtypes

Entity                                     object
Continent                                  object
Latitude                                  float64
Longitude                                 float64
Average temperature per year                int64
Hospital beds per 1000 people             float64
Medical doctors per 1000 people           float64
GDP/Capita                                float64
Population                                  int64
Median age                                  int64
Population aged 65 and over (%)             int64
Date                               datetime64[ns]
Daily tests                               float64
Cases                                     float64
Deaths                                    float64
dtype: object

In [52]:
data.isnull().sum()

Entity                                0
Continent                             0
Latitude                              0
Longitude                             0
Average temperature per year          0
Hospital beds per 1000 people         0
Medical doctors per 1000 people       0
GDP/Capita                            0
Population                            0
Median age                            0
Population aged 65 and over (%)       0
Date                                  0
Daily tests                        7895
Cases                               254
Deaths                             3610
dtype: int64

In [53]:

data = data.sort_values(by=['Entity', 'Date']).groupby('Entity').apply(lambda x: x.fillna(method='ffill')) # forward filling null values (i.e. fill null values with the values of the previous row)
data = data.sort_values(by=['Entity', 'Date']).groupby('Entity').apply(lambda x: x.fillna(method='bfill')) # backward filling null values (i.e. fill null values with the values of the following row)

C:\Users\Manika Hennedige\AppData\Local\Temp\ipykernel_35036\1374660882.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data = data.sort_values(by=['Entity', 'Date']).groupby('Entity').apply(lambda x: x.fillna(method='ffill')) # forward filling null values (i.e. fill null values with the values of the previous row)
C:\Users\Manika Hennedige\AppData\Local\Temp\ipykernel_35036\1374660882.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous beh

In [54]:
len(data["Entity"].unique()) # 104 unique countries

104

In [55]:
data["TestColumn"]  = data["Entity"]
common_cols = [] # columns that are (based on the data) dependent on the 'Entity' column
for colname in data.columns: 
    if colname != "Entity" and colname != "TestColumn":
        data["NewTestColumn"] = data["TestColumn"] + " " + data[colname].astype(str)
        if len(data["NewTestColumn"].unique()) == len(data["Entity"].unique()):
            common_cols.append(colname)
            data["TestColumn"] = data["NewTestColumn"]
        else:
            print(colname + " is not dependent on Entity")

data = data.drop(['TestColumn', 'NewTestColumn'], 1)

Date is not dependent on Entity
Daily tests is not dependent on Entity
Cases is not dependent on Entity
Deaths is not dependent on Entity


C:\Users\Manika Hennedige\AppData\Local\Temp\ipykernel_35036\3539462371.py:12: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop(['TestColumn', 'NewTestColumn'], 1)


In [56]:
common_cols # these are the columns that appear to be dependent on the Entity

['Continent',
 'Latitude',
 'Longitude',
 'Average temperature per year',
 'Hospital beds per 1000 people',
 'Medical doctors per 1000 people',
 'GDP/Capita',
 'Population',
 'Median age',
 'Population aged 65 and over (%)']

### Proposed Database Schema

#### This database schema is based on normalizing the data as much as possible to 3NF
##### Note that realistically, with a dataset spanning a longer period of time, a different schema may need to be used.
For example, if 'average temperature per year' was observed to have different values for a specific entity, an additional 'Year' column may need to be generated and associated with 'average temperature per year'. In this case, since each entity only has one unique value, I have opted for a simplified schema suitable for this data.

1. Entity
    - EntityId
    - EntityName
    - Latitude
    - Longitude
    - Average temperature per year
    - Hospital beds per 1000 people
    - Medical doctors per 1000 people
    - GDP/Capita
    - Population
    - Median age
    - Population aged 65 and over (%)
2. Continent
    - ContinentId
    - ContinentName
3. ContinentEntity
    - ContinentEntityId
    - ContinentId
    - EntityId
4. EntityDate
    - EntityDateId
    - EntityId
    - Date
    - Daily tests
    - Cases
    - Deaths

### Generating the Database
Setting IDs for Entities and Continents

In [72]:
entity_id = 1
for entity in list(data['Entity'].unique()):
    for i in range(len(data)):
        if data.loc[i, "Entity"] == entity:
            data.loc[i, 'EntityId'] = entity_id
    entity_id += 1


In [73]:
continent_id = 1
for continent in data['Continent'].unique():
    for i in range(len(data)):
        if data.loc[i, "Continent"] == continent:
            data.loc[i, 'ContinentId'] = continent_id
    continent_id += 1

In [74]:
data.head()

,Entity,Continent,Latitude,Longitude,Average temperature per year,Hospital beds per 1000 people,Medical doctors per 1000 people,GDP/Capita,Population,Median age,Population aged 65 and over (%),Date,Daily tests,Cases,Deaths,EntityId,ContinentId
0,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-25,8.0,2.0,1.0,1,1
1,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-26,5.0,2.0,1.0,1,1
2,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-27,4.0,2.0,1.0,1,1
3,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-28,1.0,2.0,1.0,1,1
4,Albania,Europe,41.15,20.17,14,2.89,1.29,5353.2,2873457,38,14,2020-02-29,8.0,2.0,1.0,1,1


In [75]:
entity = data[["EntityId", "Entity", "Latitude", "Longitude", "Average temperature per year", "Hospital beds per 1000 people", "Medical doctors per 1000 people", "GDP/Capita", "Population", "Median age", "Population aged 65 and over (%)"]]
entity = entity.drop_duplicates()

continent = data[["ContinentId", "Continent"]]
continent = continent.drop_duplicates()

continent_entity = data[["ContinentId", "EntityId"]]
continent_entity = continent_entity.drop_duplicates()
continent_entity["ContinentEntityId"] = range(1, len(continent_entity) + 1)

entity_date = data[["Date", "Daily tests", "Cases", "Deaths"]]
entity_date["EntityDateId"] = range(1, len(entity_date) + 1)

C:\Users\Manika Hennedige\AppData\Local\Temp\ipykernel_35036\4279101325.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_date["EntityDateId"] = range(1, len(entity_date) + 1)
